## Day 17

### Part 1 

As your flight slowly drifts through the sky, the Elves at the Mythical Information Bureau at the North Pole contact you. They'd like some help debugging a malfunctioning experimental energy source aboard one of their super-secret imaging satellites.

The experimental energy source is based on cutting-edge technology: a set of Conway Cubes contained in a pocket dimension! When you hear it's having problems, you can't help but agree to take a look.

The pocket dimension contains an infinite 3-dimensional grid. At every integer 3-dimensional coordinate (x,y,z), there exists a single cube which is either active or inactive.

In the initial state of the pocket dimension, almost all cubes start inactive. The only exception to this is a small flat region of cubes (your puzzle input); the cubes in this region start in the specified active (#) or inactive (.) state.

The energy source then proceeds to boot up by executing six cycles.

Each cube only ever considers its neighbors: any of the 26 other cubes where any of their coordinates differ by at most 1. For example, given the cube at x=1,y=2,z=3, its neighbors include the cube at x=2,y=2,z=2, the cube at x=0,y=2,z=3, and so on.

During a cycle, all cubes simultaneously change their state according to the following rules:

If a cube is active and exactly 2 or 3 of its neighbors are also active, the cube remains active. Otherwise, the cube becomes inactive.
If a cube is inactive but exactly 3 of its neighbors are active, the cube becomes active. Otherwise, the cube remains inactive.
The engineers responsible for this experimental energy source would like you to simulate the pocket dimension and determine what the configuration of cubes should be at the end of the six-cycle boot process.

For example, consider the following initial state:

.#.
..#
###
Even though the pocket dimension is 3-dimensional, this initial state represents a small 2-dimensional slice of it. (In particular, this initial state defines a 3x3x1 region of the 3-dimensional space.)

Simulating a few cycles from this initial state produces the following configurations, where the result of each cycle is shown layer-by-layer at each given z coordinate (and the frame of view follows the active cells in each cycle):

Before any cycles:

z=0
- .#.
- ..#
- ###


After 1 cycle:

z=-1
- #..
- ..#
- .#.

z=0
- #.#
- .##
- .#.

z=1
- #..
- ..#
- .#.


After 2 cycles:

z=-2
- .....
- .....
- ..#..
- .....
- .....

z=-1
- ..#..
- .#..#
- ....#
- .#...
- .....

z=0
- ##...
- ##...
- #....
- ....#
- .###.

z=1
- ..#..
- .#..#
- ....#
- .#...
- .....

z=2
- .....
- .....
- ..#..
- .....
- .....


After 3 cycles:

z=-2
- .......
- .......
- ..##...
- ..###..
- .......
- .......
- .......

z=-1
- ..#....
- ...#...
- #......
- .....##
- .#...#.
- ..#.#..
- ...#...

z=0
- ...#...
- .......
- #......
- .......
- .....##
- .##.#..
- ...#...

z=1
- ..#....
- ...#...
- #......
- .....##
- .#...#.
- ..#.#..
- ...#...

z=2
- .......
- .......
- ..##...
- ..###..
- .......
- .......
- .......

After the full six-cycle boot process completes, 112 cubes are left in the active state.

Starting with your given initial configuration, simulate six cycles. How many cubes are left in the active state after the sixth cycle?

In [1]:
import pandas as pd
import numpy as np

### Read in input

In [163]:
# Open file
f = open("input_data/Day17.txt", "r")

# initialize some variables
num_cycles = 6
xdim, ydim, zdim = 8,8,1
xmax, ymax, zmax = xdim+2*num_cycles, ydim+2*num_cycles, zdim+2*num_cycles

# initialize our array
active = np.zeros([xmax,ymax,zmax], dtype='bool')
neighbors = np.zeros([xmax,ymax,zmax], dtype='int')

line=f.readline()
y=0

while line:
    line = line.strip('\n')
    for i in range(len(line)):        
        active[num_cycles+y][num_cycles+i][num_cycles] = True if line[i]=='#' else False
    line=f.readline()
    y+=1
    
# Close file
f.close()

In [164]:
active.shape, neighbors.shape

((20, 20, 13), (20, 20, 13))

In [165]:
active[6][6][6]

False

### Function Definitions

In [166]:
def count_neighbors(xpos,ypos,zpos):
    '''loops through space and counts neighbors, updating the neighbors array'''

    active_neighbors = 0
    
    # take care of boundaries
    x_min = 0 if xpos==0 else xpos-1
    x_max = xmax-1 if xpos==xmax-1 else xpos+1
    y_min = 0 if ypos==0 else ypos-1
    y_max = ymax-1 if ypos==ymax-1 else ypos+1
    z_min = 0 if zpos==0 else zpos-1
    z_max = zmax-1 if zpos==zmax-1 else zpos+1
    
#    print(x_min, x_max, y_min, y_max, z_min, z_max)
    for x in range(x_min, x_max+1):
        for y in range(y_min, y_max+1):
            for z in range(z_min, z_max+1):
                # don't count itself
                if not ((x==xpos) & (y==ypos) & (z==zpos)):
                     if active[x][y][z]:
                        active_neighbors+=1
    return active_neighbors

### Main Loop

In [167]:
for i in range(1,num_cycles+1):
    
    print('Cycle: ', i)
    
    print('Counting Neighbors')   
    # go through our numpy array, counting neighbors
    for x in range(xmax):
        for y in range(ymax):
            for z in range(zmax):
                neighbors[x][y][z] = count_neighbors(x,y,z)
    
    print('Updating Status')
    # now go through our numpy array, update status
    for x in range(xmax):
        for y in range(ymax):
            for z in range(zmax):
                if active[x][y][z]:
            
                # If a cube is active, and if active neighbors are anything except 2 or 3 it becomes inactive
                    num_neighbors = neighbors[x][y][z]
                    if not ((num_neighbors == 2) | (num_neighbors == 3)):
                        active[x][y][z] = False
                        
                else: 
                
                # a cube is inactive, and if exactly three neighbors are active, it becomes active
                    if (neighbors[x][y][z] == 3):
                        active[x][y][z] = True

Cycle:  1
Counting Neighbors
Updating Status
Cycle:  2
Counting Neighbors
Updating Status
Cycle:  3
Counting Neighbors
Updating Status
Cycle:  4
Counting Neighbors
Updating Status
Cycle:  5
Counting Neighbors
Updating Status
Cycle:  6
Counting Neighbors
Updating Status


In [168]:
# Now we need to count the active states
num_active = 0
for x in range(xmax):
    for y in range(ymax):
        for z in range(zmax):
            if active[x][y][z]:
                num_active += 1
                
print('Number of active state: ', num_active)

Number of active state:  304


### Part 2

Expand to 4D.  Same rules.

In [193]:
# Open file
f = open("input_data/Day17.txt", "r")

# initialize some variables
num_cycles = 6
xdim, ydim, zdim, wdim = 8,8,1,1
xmax, ymax, zmax, wmax = xdim+2*num_cycles, ydim+2*num_cycles, zdim+2*num_cycles, wdim+2*num_cycles

# initialize our array
active = np.zeros([xmax,ymax,zmax,wmax], dtype='bool')
neighbors = np.zeros([xmax,ymax,zmax,wmax], dtype='int')

line=f.readline()
y=0

while line:
    line = line.strip('\n')
    for i in range(len(line)):        
        active[num_cycles+y][num_cycles+i][num_cycles][num_cycles] = True if line[i]=='#' else False
    line=f.readline()
    y+=1
    
# Close file
f.close()

In [194]:
active.shape, neighbors.shape

((20, 20, 13, 13), (20, 20, 13, 13))

In [195]:
active[6][7][6][6]

True

### Function Definitions

In [196]:
def count_neighbors(xpos,ypos,zpos,wpos):
    '''loops through space and counts neighbors, updating the neighbors array'''

    active_neighbors = 0
    
    # take care of boundaries
    x_min = 0 if xpos==0 else xpos-1
    x_max = xmax-1 if xpos==xmax-1 else xpos+1
  
    y_min = 0 if ypos==0 else ypos-1
    y_max = ymax-1 if ypos==ymax-1 else ypos+1
    
    z_min = 0 if zpos==0 else zpos-1
    z_max = zmax-1 if zpos==zmax-1 else zpos+1
    
    w_min = 0 if wpos==0 else wpos-1
    w_max = wmax-1 if wpos==wmax-1 else wpos+1
    
    
#    print(x_min, x_max, y_min, y_max, z_min, z_max)
    for x in range(x_min, x_max+1):
        for y in range(y_min, y_max+1):
            for z in range(z_min, z_max+1):
                for w in range(w_min, w_max+1):
                    # don't count itself
                    if not ((x==xpos) & (y==ypos) & (z==zpos) & (w==wpos)):
                         if active[x][y][z][w]:
                            active_neighbors+=1
    return active_neighbors

### Main Loop

In [197]:
for i in range(1,num_cycles+1):
    
    print('Cycle: ', i)
    
    print('Counting Neighbors')   
    # go through our numpy array, counting neighbors
    for x in range(xmax):
        for y in range(ymax):
            for z in range(zmax):
                for w in range(wmax):
                    neighbors[x][y][z][w] = count_neighbors(x,y,z,w)
    
    print('Updating Status')
    # now go through our numpy array, update status
    for x in range(xmax):
        for y in range(ymax):
            for z in range(zmax):
                for w in range(wmax):
                    if active[x][y][z][w]:
            
                    # If a cube is active, and if active neighbors are anything except 2 or 3 it becomes inactive
                        num_neighbors = neighbors[x][y][z][w]
                        if not ((num_neighbors == 2) | (num_neighbors == 3)):
                            active[x][y][z][w] = False
                        
                    else: 
                
                    # a cube is inactive, and if exactly three neighbors are active, it becomes active
                        if (neighbors[x][y][z][w] == 3):
                            active[x][y][z][w] = True

Cycle:  1
Counting Neighbors
Updating Status
Cycle:  2
Counting Neighbors
Updating Status
Cycle:  3
Counting Neighbors
Updating Status
Cycle:  4
Counting Neighbors
Updating Status
Cycle:  5
Counting Neighbors
Updating Status
Cycle:  6
Counting Neighbors
Updating Status


In [198]:
# Now we need to count the active states
num_active = 0
for x in range(xmax):
    for y in range(ymax):
        for z in range(zmax):
            for w in range(wmax):
                if active[x][y][z][w]:
                    num_active += 1
                
print('Number of active state: ', num_active)

Number of active state:  1868
